In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

# importo los .py
import sys
sys.path.append("aux")
import config

from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
arch_entrada = "datasets/emp3_sellout_transf.csv"
arch_salida = "datasets/emp3_sellout_train.csv"

arch_productos = "datasets/emp3_sellout_producto_base.csv"
arch_clientes = "datasets/emp3_sellout_cliente_base.csv"
arch_cat1 = "datasets/emp3_sellout_cat1_base.csv"
arch_cat2 = "datasets/emp3_sellout_cat2_base.csv"
arch_cat3 = "datasets/emp3_sellout_cat3_base.csv"

In [4]:
def crear_nuevos_features(df_param):
    # Diferencia entre lo solicitado y lo entregado
    df_param["dif_cust_request_tn"]=df_param.cust_request_tn-df_param.tn
    df_param["dif_cust_request_tn_porc"]=np.where(df_param.cust_request_tn==0,0,100*df_param.dif_cust_request_tn/df_param.cust_request_tn)
   
    return df_param

In [5]:
def crear_features_temporales(campo,df_param):
    # Primero y FUNDAMENTAL, ordeno por el campo y periodo
    df_param = df_param.sort_values(by=[campo,"periodo"],ascending=True)

    if campo=="prod_cust":
        prefijo_tn = ""
    else:
        prefijo_tn = campo + "_"
    for i in range(1, config.NUM_LAGS_PARAM + 1):
        df_param[campo + "_tn_lag_" + str(i)] = df_param.groupby(campo)[prefijo_tn + "tn"].shift(i)
        df_param[campo + "_tn_delta_" + str(i)] = df_param.groupby(campo)[prefijo_tn + "tn"].diff(i)
        df_param[campo + "_rolling_std_" + str(i)] = df_param.groupby(campo)[prefijo_tn + "tn"].rolling(window=i).std().reset_index(level=0, drop=True)
        df_param[campo + "_rolling_mean_" + str(i)] = df_param.groupby(campo)[prefijo_tn + "tn"].rolling(i).mean().reset_index(level=0, drop=True)
        df_param[campo + "_rolling_sum_" + str(i)] = df_param.groupby(campo)[prefijo_tn + "tn"].rolling(window=i).sum().reset_index(level=0, drop=True)

    return df_param

In [6]:
df_sellout = pd.read_csv(arch_entrada)
df_productos = pd.read_csv(arch_productos)
df_clientes = pd.read_csv(arch_clientes)
df_cat1 = pd.read_csv(arch_cat1)
df_cat2 = pd.read_csv(arch_cat2)
df_cat3 = pd.read_csv(arch_cat3)

In [7]:
df_sellout = crear_nuevos_features(df_sellout)
df_sellout = crear_features_temporales("prod_cust",df_sellout)

df_productos = crear_features_temporales("product_id",df_productos)
df_clientes = crear_features_temporales("customer_id",df_clientes)
df_cat1 = crear_features_temporales("cat1",df_cat1)
df_cat2 = crear_features_temporales("cat2",df_cat2)
df_cat3 = crear_features_temporales("cat3",df_cat3)

print(len(df_sellout))
df_sellout = df_sellout.merge(df_productos,how="left",on=["product_id","periodo"])
df_sellout = df_sellout.merge(df_clientes,how="left",on=["customer_id","periodo"])
df_sellout = df_sellout.merge(df_cat1,how="left",on=["cat1","periodo"])
df_sellout = df_sellout.merge(df_cat2,how="left",on=["cat2","periodo"])
df_sellout = df_sellout.merge(df_cat3,how="left",on=["cat3","periodo"])

print(len(df_sellout))



4020
4020


In [8]:
df_sellout.head(100)

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_fecha,mes,...,cat3_tn_lag_14,cat3_tn_delta_14,cat3_rolling_std_14,cat3_rolling_mean_14,cat3_rolling_sum_14,cat3_tn_lag_15,cat3_tn_delta_15,cat3_rolling_std_15,cat3_rolling_mean_15,cat3_rolling_sum_15
0,20001,201701,10001,0,0.0,11.0,0.343662,0.343662,2017-01-01,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,201702,10001,0,0.0,23.0,0.687207,0.687207,2017-02-01,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,201703,10001,0,0.0,33.0,0.319562,0.319562,2017-03-01,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20001,201704,10001,0,0.0,8.0,0.045956,0.045956,2017-04-01,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20001,201705,10001,0,0.0,15.0,0.349774,0.349077,2017-05-01,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20001,201812,10003,0,0.0,23.0,0.358943,0.358943,2018-12-01,12,...,5215.16209,-369.89866,822.238318,5175.651712,72459.12397,5299.00027,-453.73684,792.394340,5178.285737,77674.28606
96,20001,201901,10003,0,0.0,23.0,0.657724,0.655041,2019-01-01,1,...,4920.63610,-205.92089,828.965377,5160.943077,72253.20308,5215.16209,-500.44688,801.217126,5144.922612,77173.83918
97,20001,201902,10003,0,0.0,18.0,0.372967,0.294715,2019-02-01,2,...,4434.73833,368.62770,809.756306,5187.273627,72621.83078,4920.63610,-117.27007,804.128810,5137.104607,77056.56911
98,20001,201903,10003,0,0.0,18.0,0.786057,0.620163,2019-03-01,3,...,4549.79830,955.07698,791.955198,5255.493411,73576.90776,4434.73833,1070.13695,784.597926,5208.447071,78126.70606


In [9]:
df_sellout.to_csv(arch_salida, index=False)

## Verificaciones

In [10]:
# Nuevos features
df_control = df_sellout[df_sellout.dif_cust_request_tn_porc > 0]
df_control[["product_id","customer_id","periodo","tn","cust_request_tn","dif_cust_request_tn","dif_cust_request_tn_porc"]].sample(10)

,product_id,customer_id,periodo,tn,cust_request_tn,dif_cust_request_tn,dif_cust_request_tn_porc
909,20003,10002,201710,0.492747,0.494649,0.001902,0.384615
2336,20006,10005,201909,0.263415,0.328455,0.065041,19.801978
2635,20007,10002,201708,0.075989,0.119804,0.043814,36.571825
3089,20009,10002,201906,1.663803,1.833906,0.170103,9.275456
923,20003,10002,201812,0.377705,0.502319,0.124614,24.807662
3067,20009,10002,201708,0.336990,0.392244,0.055254,14.086555
3428,20009,10013,201709,0.958678,1.117768,0.159091,14.232890
3266,20009,10007,201903,1.449616,1.783108,0.333492,18.702863
506,20002,10003,201703,0.939570,0.940001,0.000431,0.045883
2647,20007,10002,201808,0.772947,0.810526,0.037579,4.636369


In [11]:
# Lags prod-cust
df_sellout[["prod_cust","periodo","tn","prod_cust_tn_lag_1","prod_cust_tn_lag_2","prod_cust_tn_lag_3"]].head(50)

,prod_cust,periodo,tn,prod_cust_tn_lag_1,prod_cust_tn_lag_2,prod_cust_tn_lag_3
0,20001-10001,201701,0.343662,NaN,NaN,NaN
1,20001-10001,201702,0.687207,0.343662,NaN,NaN
2,20001-10001,201703,0.319562,0.687207,0.343662,NaN
3,20001-10001,201704,0.045956,0.319562,0.687207,0.343662
4,20001-10001,201705,0.349077,0.045956,0.319562,0.687207
5,20001-10001,201706,0.442536,0.349077,0.045956,0.319562
6,20001-10001,201707,0.349774,0.442536,0.349077,0.045956
7,20001-10001,201708,0.149781,0.349774,0.442536,0.349077
8,20001-10001,201709,1.000000,0.149781,0.349774,0.442536
9,20001-10001,201710,0.767630,1.000000,0.149781,0.349774


In [12]:
# Delta Lags prod-cust
#df_sellout[["prod_cust","periodo","tn","tn_lag_1","tn_delta_1","tn_lag_2","tn_delta_2","tn_lag_3","tn_delta_3"]].head(50)

In [13]:
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4020 entries, 0 to 4019
Columns: 485 entries, product_id to cat3_rolling_sum_15
dtypes: bool(1), float64(470), int64(8), object(6)
memory usage: 14.9+ MB


In [14]:
df_sellout.isna().sum()

product_id                  0
periodo                     0
customer_id                 0
cero_ventas                 0
plan_precios_cuidados       0
                         ... 
cat3_tn_lag_15           1620
cat3_tn_delta_15         1620
cat3_rolling_std_15      1512
cat3_rolling_mean_15     1512
cat3_rolling_sum_15      1512
Length: 485, dtype: int64

In [15]:
df_sellout.columns

Index(['product_id', 'periodo', 'customer_id', 'cero_ventas',
       'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'tn',
       'periodo_fecha', 'mes',
       ...
       'cat3_tn_lag_14', 'cat3_tn_delta_14', 'cat3_rolling_std_14',
       'cat3_rolling_mean_14', 'cat3_rolling_sum_14', 'cat3_tn_lag_15',
       'cat3_tn_delta_15', 'cat3_rolling_std_15', 'cat3_rolling_mean_15',
       'cat3_rolling_sum_15'],
      dtype='object', length=485)

In [16]:
len(df_sellout.columns)

485